In [1654]:
try:
  import google.colab
  from google.colab import files
  IN_COLAB = True
except:
  IN_COLAB = False

In [1655]:
!pip install alive_progress
import pandas as pd
import math
import io
from datetime import datetime, time
from alive_progress import alive_bar
import os


# Transcript Mapping

### Set File Names and Variables

Set file names here, the upload boxes will appear under each section of the Read Data section. Please make sure that the real filenames match the file names entered here. 

In [1656]:
# Controls if a sample will be used
# Set to [RANDOM, HEAD, TAIL, NONE]
FILTER = "RANDOM"
SIZE = 1000

# Training Records from SumTotal
filename = "2020-2022 Training Records.xlsx"

# Coursename Mapping Setup
USE_COURSE_MAP = True
CMAP_ORIG = "OLD_CNAME"
CMAP_NEW = "NEW_CNAME"
MAP_PATH = "map.csv"

# Saba Courses
COURSE_PATH = "saba_courses.csv"

# Saba People
PEOPLE_PATH = "saba_people.csv"

### Other Global Variables

In [1657]:
USERNAME = "Employee Number"
COURSEID = "OFFERING"
COURSEVERSION = "COURSE_VERSION"
COMPDATE = "Completion Date"
STATUS = "Completion Status"
COURSETITLE = "Activity Name"
PASSFAIL = "Pass/Fail"
ATTSTATUS = "Attendance Status"
ACT_TYPE = "Activity Type"

FOLDER = "trans_output/"
OUTPUT_PRE = "transcript_out_"
ERRORS_PRE = "transcript_errors_"
MISSING_COURSE = "missing_course_"
MISSING_PEOPLE = "missing_people_"
FILE_EXT = ".csv"
TIME_SUFF = datetime.now().strftime('%Y_%m_%d-%H-%M')

## Read Data

#### Transcript List

In [1658]:
### Note: if using Google Colab, this must be set to the filename of the 
# file you will load. This is a quirk of Colab. 
if IN_COLAB:
  print("Colab")
  uploaded = files.upload()
  source = pd.read_excel(io.BytesIO(uploaded.get(filename)))
else:
  print("Not Colab")
  source = pd.read_excel(filename)
#source = source[source[ACT_TYPE] == "Course"]

Not Colab


#### Mapping Data

In [1659]:
def map_course(mapping_list, orig_df):
    new_df = orig_df
    for _, map_row in mapping_list.iterrows():
        old_course = map_row[CMAP_ORIG]
        new_course = map_row[CMAP_NEW]
        #new_df[[COURSEID]] = new_df[[COURSEID]].replace(old_course, new_course)
        new_df.replace(old_course, new_course, inplace=True)
        #print(old_course, new_course)
    return new_df

In [1660]:
if USE_COURSE_MAP:
  if IN_COLAB:
    print("Colab")
    uploaded = files.upload()
    course_map = pd.read_csv(io.BytesIO((uploaded.get(MAP_PATH))))
  else:
    print("Not Colab")
    course_map = pd.read_csv(MAP_PATH)

Not Colab


#### Saba Courses

In [1661]:
if IN_COLAB:
  print("Colab")
  uploaded = files.upload()
  courses = pd.read_csv(io.BytesIO((uploaded.get(COURSE_PATH))))
else:
  print("Not Colab")
  courses = pd.read_csv(COURSE_PATH)


Not Colab


#### Saba People

In [1662]:
if IN_COLAB:
  print("Colab")
  uploaded = files.upload()
  people = pd.read_csv(io.BytesIO((uploaded.get(PEOPLE_PATH))))
else:
  print("Not Colab")
  people = pd.read_csv(PEOPLE_PATH)

Not Colab


## Begin Processing

In [1663]:
# Generate Output Filename
def genFileName(prefix, suffix, extension):
    cwd = os.getcwd()+"/"
    return cwd+FOLDER+str(prefix)+str(suffix)+str(extension)

In [1664]:
# Take Data Sample if Testing
if FILTER == "RANDOM":
    source = source.sample(SIZE)
elif FILTER == "HEAD":
    source = source.head(SIZE)
elif FILTER == "TAIL":
    source = source.tail(SIZE)


In [1665]:
source.head(5)

,Employee Number,Employee Name,Waived Notes,Employee email,Job Title,Department Code,Department Name,MANAGER,Activity Type,OFFERING,Activity Name,COURSE_VERSION,Score,Attendance Status,Pass/Fail,Completion Status,Completion Date,Expiration Date,Note
95686,91954,"LOHNERT, Leanne",NaN,leanne.lohnert@cnl.ca,Health & Safety Field Speciali,8590.0,"HWP SAFETY, SECURITY & EMERG P","MANIC, Serge",ILT Course,RP-G3,Group 3 - Radiation Protection Training,NaN,90.0,Attended,Pass,Completed,2021-01-12,2025-01-12,NaN
38897,57857,"LAMARCHE, Chris",NaN,Chris.Lamarche@cnl.ca,S&T (UC) Licensed Facilities S,8659.0,LICENSED NUCLEAR FACILITIES,"WILLIAMS, Jonathan",Course,OSH-9045-Online (Rev 1),Cost Course - Hand Safety (1.0),1.0,100.0,Attended,Pass,Completed,2021-07-16,NaT,NaN
23095,55136,"KERR, Andrew",ATOM ID 54381126,Andrew.Kerr@cnl.ca,Decommissioning Specialist,8270.0,CRL DECOMM PROGRAM EXECUTION,"SIMARD, Dan",ILT Class,MS-1002,Waiver for Integrated Work Control - Enhanced,NaN,NaN,Waived,Pass,Completed,2021-04-07,NaT,NaN
63324,60949,"BROWN, Scott",NaN,Scott.Brown@cnl.ca,Section Leader,8270.0,CRL DECOMM PROGRAM EXECUTION,"AIKENS, Patrick Joseph",Versional,LGL-1003-Online,Values and Ethics at CNL,NaN,100.0,Attended,Pass,Completed,2021-11-15,2022-11-16,NaN
49077,59995,"OLSHESKIE, Crystal",NaN,crystal.olsheskie@cnl.ca,Administrative Assistant,8663.0,ADVANCED REACTORS DIRECTORATE,"HAM-SU, Rosaura",Versional,OSH-9045-Online,Hand Safety,NaN,91.0,Attended,Pass,Completed,2021-10-27,NaT,NaN


### Generate Status Table (Optional)

In [1666]:
#print(source.shape[1])
#source[["Completion Status", "Attendance Status", "Pass/Fail"]].groupby(["Pass/Fail","Completion Status", "Attendance Status"], dropna=False, as_index=False).size()

### Update Mapped Course Names

In [1667]:
if USE_COURSE_MAP:
    source = map_course(course_map, source)
source.head(5)

,Employee Number,Employee Name,Waived Notes,Employee email,Job Title,Department Code,Department Name,MANAGER,Activity Type,OFFERING,Activity Name,COURSE_VERSION,Score,Attendance Status,Pass/Fail,Completion Status,Completion Date,Expiration Date,Note
95686,91954,"LOHNERT, Leanne",NaN,leanne.lohnert@cnl.ca,Health & Safety Field Speciali,8590.0,"HWP SAFETY, SECURITY & EMERG P","MANIC, Serge",ILT Course,RP-G3,Group 3 - Radiation Protection Training,NaN,90.0,Attended,Pass,Completed,2021-01-12,2025-01-12,NaN
38897,57857,"LAMARCHE, Chris",NaN,Chris.Lamarche@cnl.ca,S&T (UC) Licensed Facilities S,8659.0,LICENSED NUCLEAR FACILITIES,"WILLIAMS, Jonathan",Course,OSH-9045-Online (Rev 1),Cost Course - Hand Safety (1.0),1.0,100.0,Attended,Pass,Completed,2021-07-16,NaT,NaN
23095,55136,"KERR, Andrew",ATOM ID 54381126,Andrew.Kerr@cnl.ca,Decommissioning Specialist,8270.0,CRL DECOMM PROGRAM EXECUTION,"SIMARD, Dan",ILT Class,MS-1002,Waiver for Integrated Work Control - Enhanced,NaN,NaN,Waived,Pass,Completed,2021-04-07,NaT,NaN
63324,60949,"BROWN, Scott",NaN,Scott.Brown@cnl.ca,Section Leader,8270.0,CRL DECOMM PROGRAM EXECUTION,"AIKENS, Patrick Joseph",Versional,LGL-1003-Online,Values and Ethics at CNL,NaN,100.0,Attended,Pass,Completed,2021-11-15,2022-11-16,NaN
49077,59995,"OLSHESKIE, Crystal",NaN,crystal.olsheskie@cnl.ca,Administrative Assistant,8663.0,ADVANCED REACTORS DIRECTORATE,"HAM-SU, Rosaura",Versional,OSH-9045-Online,Hand Safety,NaN,91.0,Attended,Pass,Completed,2021-10-27,NaT,NaN


### Read SABA Data

#### Course Data

In [1668]:
# Process Course Data from Saba
course_id = courses[["Course ID", "Version"]]
ids_only = pd.Series(courses["Course ID"].unique())

courses.head()

,Title,Version,Course ID,Unnamed: 3
0,100035-IBM Cognos Analytics Advanced (Cost Cou...,1.0,IT-9050,NaN
1,100035-IBM Cognos Analytics Fundamentals (Cost...,1.0,IT-9049,NaN
2,10 Commandments of Training,1.0,TD-1048,NaN
3,12 GA Shotgun,1.0,SECU-3015,NaN
4,5S Methodology (Cost Course),1.0,PI-9006-ONLINE,NaN


#### Employee Data

In [1669]:
# Process People Data from Saba
people_un = set(people["Username"].astype("int64", errors="ignore").unique())
#people.head()
len(people_un)

3869

### Data Processing Functions

In [1670]:
def procStatus(pass_fail, comp_status, att_status):
    status = -1
    ## Fail
    if pass_fail == "Fail":
        status = 400
    elif (pass_fail == "Pass") & (comp_status == "Not Completed"):
        status = 400
    #elif (math.isnan(pass_fail)) & (math.isnan(comp_status)) & ((att_status == "No-show") | (att_status == "Replaced")):
    #    status = 400
    ## Pass
    elif (pass_fail == "Pass") & (comp_status != "Not Completed"):
        status = 200
    elif (pd.isnull(pass_fail)) & (comp_status == "Completed"):
        status = 200
        #print("no p/f, status is complete")
    elif ( (pd.isnull(pass_fail)) & (pd.isnull(comp_status)) & (att_status == "Attended") ):
        status = 200
        #print("no p/f, no status, attstatus is attended")
    else:
        status = 400
    return status

def checkPerson(unList, person):
    is_in = False
    if str(person) in unList:
        is_in = True
    return is_in

def checkCourse(courseList, courseID, version):
    is_in = False
    replace_version = version
    for _, row in courseList.iterrows():
        #print(courseID, version, row["Course ID"], row["Version"])
        if (courseID == row["Course ID"]):
            #print("Course Match", courseID, version, row["Course ID"], row["Version"])
            if pd.isnull(version):
                replace_version = row["Version"]
                #print("Insert Version", replace_version, version)
            #else:
                #print("Orig Version:", replace_version, version)
            is_in = True
            #print("Match:", courseID, replace_version, version)
    return is_in, replace_version

class TranscriptRow:
    def __init__(self, username, courseID, courseVersion, compDate, status, courseTitle, inCourse, inPerson):
        self.username = username
        self.courseID = courseID.upper()
        self.courseVersion = courseVersion
        self.compDate = compDate
        self.status = status
        self.courseTitle = courseTitle
        self.inCourse = inCourse
        self.inPerson = inPerson


In [1671]:
# Process Data
source_used = source[[USERNAME, COURSEID, COURSEVERSION, COMPDATE, PASSFAIL, STATUS, ATTSTATUS, COURSETITLE]]

transcripts = []

with alive_bar(source_used.shape[0], force_tty=True) as bar_read:
    for index, trans in source_used.iterrows():
        username = trans[USERNAME]
        courseID = trans[COURSEID].upper()
        course_ver = trans[COURSEVERSION]
        comp_date = trans[COMPDATE].strftime('%Y-%m-%d')
        status = procStatus(trans[PASSFAIL], trans[STATUS], trans[ATTSTATUS])
        course_title = trans[COURSETITLE]
        in_course, rep_ver = checkCourse(course_id, courseID, course_ver)
        in_pers = checkPerson(people_un, username)

        #Update Version
        course_ver = rep_ver
        
        trans_entry = TranscriptRow(username, courseID, course_ver, comp_date, status, course_title, in_course, in_pers)
        transcripts.append(trans_entry)
        #print(courseID, type(courseID), courseID in ids_only.values)
        bar_read()
print("Transcripts Processed:", len(transcripts))

|████████████████████████████████████████| 1000/1000 [100%] in 1:02.1 (16.11/s)                                          ▅▃▁ 22/1000 [2%] in 1s (16.9/s, eta: 59s)                                    ▄▆█ 65/1000 [6%] in 4s (16.6/s, eta: 56s) ▃▅▇ 114/1000 [11%] in 7s (16.1/s, eta: 55s) (15.4/s, eta: 56s) ▆▄▂ 153/1000 [15%] in 10s (15.7/s, eta: 54s) in 10s (15.8/s, eta: 54s)  ▁▃▅ 254/1000 [25%] in 15s (16.5/s, eta: 46s) (16.5/s, eta: 43s) ▅▇▇ 293/1000 [29%] in 18s (16.6/s, eta: 43s) 377/1000 [38%] in 23s (16.6/s, eta: 38s)  ▄▂▂ 411/1000 [41%] in 25s (16.5/s, eta: 36s)  ▄▆█ 420/1000 [42%] in 26s (16.4/s, eta: 35s)  ▅▇▇ 443/1000 [44%] in 27s (16.2/s, eta: 34s)  ▄▆█ 466/1000 [47%] in 29s (16.1/s, eta: 33s)  ▂▄▆ 607/1000 [61%] in 37s (16.3/s, eta: 24s) (16.3/s, eta: 22s) ▃▅▇ 668/1000 [67%] in 41s (16.3/s, eta: 20s) 736/1000 [74%] in 45s (16.4/s, eta: 16s)  ▃▁▃ 746/1000 [75%] in 45s (16.4/s, eta: 16s) (16.4/s, eta: 14s) (16.4/s, eta: 13s) ▇▇▅ 876/1000 [88%] in 54s (16.4/s, eta: 8s) in 54s (16.4

### Print to File Outputs

In [1672]:
# Process a Transcript Row
def write_row(df_output, trans, test=False, subset="ALL"):
    #df = df_output
    
    inCourse = trans.inCourse
    inPerson = trans.inPerson

    tmp_dict = {
        "LEARNER": trans.username,
        "COURSE": trans.courseID,
        "COURSE_VERSION": trans.courseVersion,
        "COMPLETION_DATE": trans.compDate,
        "COMPLETION_STATUS": trans.status,
        "CREATE_ADHOC_COURSE": "FALSE",
        "COURSE_TITLE": trans.courseTitle
    }
    if test == True:
        tmp_dict = {
            "LEARNER": trans.username,
            "COURSE": trans.courseID,
            "COURSE_VERSION": trans.courseVersion,
            "COMPLETION_DATE": trans.compDate,
            "COMPLETION_STATUS": trans.status,
            "CREATE_ADHOC_COURSE": "FALSE",
            "COURSE_TITLE": trans.courseTitle,
            "IN_COURSE": trans.inCourse,
            "IN_PERSON": trans.inPerson
        }
    if subset == "ALL":
        df_output = df_output.append(tmp_dict, ignore_index=True)
    elif subset == "CLEAN":
        if ( (inCourse == 1) & (inPerson == 1) ):
            df_output = df_output.append(tmp_dict, ignore_index=True)
    elif subset == "MISSING":
        if ( (inCourse == 0) | (inPerson == 0) ):
            tmp_dict["COURSE_GOOD"] = bool(inCourse)
            tmp_dict["PERSON_GOOD"] = bool(inPerson)
            df_output = df_output.append(tmp_dict, ignore_index=True)
    
    return df_output
    

In [1673]:
# Output Header Values
out_header = [
    "LEARNER",
    "COURSE",
    "COURSE_VERSION",
    "COMPLETION_DATE",
    "COMPLETION_STATUS",
    "CREATE_ADHOC_COURSE",
    "COURSE_TITLE"
]

### Generate Saba Import Data Table and Write to File

In [1674]:
out = pd.DataFrame(columns = out_header)

with alive_bar(len(transcripts), force_tty=True) as bar:
    for ts in transcripts:
        out = write_row(out, ts, False, "CLEAN")
        bar()
out["ID"] = out.index
out.sample(20).T

|████████████████████████████████████████| 1000/1000 [100%] in 2.2s (456.11/s)                                          , eta: 1s)  ▆█▆ 657/1000 [66%] in 2s (434.2/s, eta: 1s) 


,263,509,36,229,90,65,516,67,318,375,332,373,172,428,19,313,151,364,48,252
LEARNER,92361,57053,92329,92665,55646,92313,91132,90381,92405,54276,91588,90174,52382,61141,90242,90045,92098,90077,56045,61806
COURSE,OSH-9104-ONLINE,TD-2000-WL,RP-G1-014,OSH-1004-VIRTUAL,TD-2000-WL,HR-1005,FIRE-1001-ONLINE,HR-1005,OSH-1003-ONLINE,NMMS-1002,HR-1005,MS-1002,FIRE-3001,OSH-9045-ONLINE,PMO-9013,FIRE-1043-CRL,SECU-4013,FIRE-3008-CRL,OSH-3005,HR-1005
COURSE_VERSION,1.0,1.0,1.0,1.0,1.0,1.0,4.7,1.0,1.6,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
COMPLETION_DATE,2021-10-18,2021-08-23,2021-09-20,2022-03-23,2021-09-28,2021-11-08,2022-01-18,2021-12-06,2021-11-23,2021-11-29,2021-11-10,2021-04-07,2021-10-27,2021-10-27,2021-06-01,2021-03-29,2021-05-07,2021-05-10,2022-01-13,2022-01-25
COMPLETION_STATUS,200,200,200,200,200,200,200,200,200,400,200,200,200,400,200,200,200,200,200,400
CREATE_ADHOC_COURSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
COURSE_TITLE,Cost Course - Trenching Safety Theory,GET General Employee Training,Group 1 - Radiation Detector Theory,VIRTUAL Cost Course - $222/Person - Lock Out /...,GET General Employee Training,Virtual Mandatory Harassment and Violence Prev...,Fire Prevention,Virtual Mandatory Harassment and Violence Prev...,Aerial Work Platform Theory,VIRTUAL Nuclear Criticality Safety,Mandatory Harassment and Violence Prevention T...,Waiver for Integrated Work Control - Enhanced,Private Offering of Fire Extinguisher - Practical,Hand Safety,Cost Course - $495/person VIRTUAL Project Sche...,Fire Control: Theory: Suppressing Class D Fires,B700 Reception Counter,Confined Space Entry: Practical,WS BLOCK - Working at Heights Practical,Mandatory Harassment and Violence Prevention T...
ID,263,509,36,229,90,65,516,67,318,375,332,373,172,428,19,313,151,364,48,252


In [1675]:
if IN_COLAB:
    out.to_csv("out_transcript.csv", index=False)
else:
    out.to_csv(genFileName(OUTPUT_PRE, TIME_SUFF, FILE_EXT), index=False)

In [1676]:
# OLD OUTPUT
#print(out.shape)
#out.to_csv("out_transcript.csv", index=False)
#out.to_csv(genFileName(OUTPUT_PRE, TIME_SUFF, FILE_EXT), index=False)

#### Write Failiures to File

In [1677]:
out_err = pd.DataFrame(columns = out_header)

with alive_bar(len(transcripts), force_tty=True) as bar:
    for ts in transcripts:
        out_err = write_row(out_err, ts, False, "MISSING")
        bar()
out_err["ID"] = out_err.index
out_err.sample(20).T

|████████████████████████████████████████| 1000/1000 [100%] in 1.5s (670.82/s)                                          s, eta: 1s) 


,204,292,346,210,264,405,321,384,154,37,336,386,233,133,363,88,112,327,220,208
LEARNER,59286,27606,60409,91266,58693,54886,60035,92106,91214,91992,60038,CONT-3099,92353,59114,52815,60800,61035,20247,59370,CONT-2844
COURSE,FIRE-1001-ONLINE (REV 4.5),ENG-1069-ONLINE (REV 0.1),OSH-9045-ONLINE (REV 1),OSH-9045-ONLINE (REV 1),LGL-1003-ONLINE,SECU-2008-ONLINE (REV 1.2),OSH-1006,PROC-1019,OSH-1004-ONLINE,RP-G4-ONLINE,WL-315,MS-1004-WL,QUAL-1013-ONLINE (REV 3),OSH-9045-ONLINE (REV 1),FD-2001-B204,SECU-2008-ONLINE (REV 1.2),OSH-1001-ONLINE (REV 1.2),SECU-2008-ONLINE (REV 1.3),OSH-9045-ONLINE (REV 1),WL-120
COURSE_VERSION,4.5,0.1,1.0,1.0,NaN,1.2,NaN,1.0,NaN,NaN,1.0,1.0,3.0,1.0,NaN,1.2,1.2,1.3,1.0,1.0
COMPLETION_DATE,2021-06-21,2022-03-29,2021-10-13,2021-07-16,2021-12-10,2021-05-06,2021-02-10,2021-05-20,2021-01-14,2021-07-12,2021-03-11,2022-01-10,2021-11-01,2021-11-04,2021-02-15,2021-05-19,2021-02-18,2021-06-03,2021-07-26,2021-05-03
COMPLETION_STATUS,200,200,200,200,200,200,200,200,200,200,400,200,400,200,200,200,200,200,200,200
CREATE_ADHOC_COURSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
COURSE_TITLE,Fire Prevention (4.5),Fume Hood Safety (0.1),Cost Course - Hand Safety (1.0),Cost Course - Hand Safety (1.0),Values and Ethics at CNL,Security Awareness Refresher (1.2),Confined Space Entry,VIRTUAL CNL Contractor Management Overview,Lock Out Tag Out Exam,Group 4 Radiation Protection Training,Respectful Work Place,Integrated Work Control 5 Core Elements,FME Awareness CRL (3.0),Cost Course - Hand Safety (1.0),B204 Building Familiarization Checkout,Security Awareness Refresher (1.2),Crane - Safe Indoor Hoist - Theory (1.2),Security Awareness Refresher (1.3),Cost Course - Hand Safety (1.0),Whiteshell Site Orientation
COURSE_GOOD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
PERSON_GOOD,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
ID,204,292,346,210,264,405,321,384,154,37,336,386,233,133,363,88,112,327,220,208


In [1678]:
if IN_COLAB:
    out_err.to_csv("out_errors.csv", index=False)
else:
    out_err.to_csv(genFileName(ERRORS_PRE, TIME_SUFF, FILE_EXT), index=False)

In [1679]:
# OLD OUTPUT
#print(out_err.shape)
#out_err.to_csv("out_errors.csv", index=False)
#out_err.to_csv(genFileName(ERRORS_PRE, TIME_SUFF, FILE_EXT), index=False)

### Generate Lists of Missing Courses and People

Generate list of course IDs and usernames that are in the transcript data, but do not exist in Saba. We know these will fail since the requisite data does not exist in Saba. 

In [1680]:
course_need = out_err[out_err["COURSE_GOOD"] == False]
person_need = out_err[out_err["PERSON_GOOD"] == False]

# Old Output
#pd.Series(course_need["COURSE"].unique(), name="Missing Courses").to_csv("out_courses.csv")
#pd.Series(person_need["LEARNER"].unique(), name="Missing Learners").to_csv("out_people.csv")
#pd.Series(course_need["COURSE"].unique(), name="Missing Courses").to_csv(genFileName(MISSING_COURSE, TIME_SUFF, FILE_EXT))
#pd.Series(person_need["LEARNER"].unique(), name="Missing Learners").to_csv(genFileName(MISSING_PEOPLE, TIME_SUFF, FILE_EXT))

In [1681]:
if IN_COLAB:
    pd.Series(course_need["COURSE"].unique(), name="Missing Courses").to_csv("out_courses.csv")
    pd.Series(person_need["LEARNER"].unique(), name="Missing Learners").to_csv("out_people.csv")
else:
    pd.Series(course_need["COURSE"].unique(), name="Missing Courses").to_csv(genFileName(MISSING_COURSE, TIME_SUFF, FILE_EXT))
    pd.Series(person_need["LEARNER"].unique(), name="Missing Learners").to_csv(genFileName(MISSING_PEOPLE, TIME_SUFF, FILE_EXT))